Hola **Daniel**!

Soy **Patricio Requena** 👋. Es un placer ser el revisor de tu proyecto el día de hoy!

Revisaré tu proyecto detenidamente con el objetivo de ayudarte a mejorar y perfeccionar tus habilidades. Durante mi revisión, identificaré áreas donde puedas hacer mejoras en tu código, señalando específicamente qué y cómo podrías ajustar para optimizar el rendimiento y la claridad de tu proyecto. Además, es importante para mí destacar los aspectos que has manejado excepcionalmente bien. Reconocer tus fortalezas te ayudará a entender qué técnicas y métodos están funcionando a tu favor y cómo puedes aplicarlos en futuras tareas. 

_**Recuerda que al final de este notebook encontrarás un comentario general de mi parte**_, empecemos!

Encontrarás mis comentarios dentro de cajas verdes, amarillas o rojas, ⚠️ **por favor, no muevas, modifiques o borres mis comentarios** ⚠️:


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si todo está perfecto.
</div>

<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si tu código está bien pero se puede mejorar o hay algún detalle que le hace falta.
</div>

<div class="alert alert-block alert-danger">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si de pronto hace falta algo o existe algún problema con tu código o conclusiones.
</div>

Puedes responderme de esta forma:
<div class="alert alert-block alert-info">
<b>Respuesta del estudiante</b> <a class=“tocSkip”></a>
Muchísimas gracias por las observaciones 🤗
</div>

# Paso 1: Abre y examina el archivo de datos. Dirección al archivo:/datasets/users_behavior.csv

In [186]:
# Cargar libería pandas
import pandas as pd

In [187]:
# Importar el dataset

df_users_behavior = pd.read_csv('/datasets/users_behavior.csv')

In [188]:
# Estudiar los datos que contienen

df_users_behavior.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [189]:
#Asegurarse de que los tipos de datos sean correctos
df_users_behavior.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [190]:
# Explorar estadística descriptiva
print(df_users_behavior.describe())

             calls      minutes     messages       mb_used     is_ultra
count  3214.000000  3214.000000  3214.000000   3214.000000  3214.000000
mean     63.038892   438.208787    38.281269  17207.673836     0.306472
std      33.236368   234.569872    36.148326   7570.968246     0.461100
min       0.000000     0.000000     0.000000      0.000000     0.000000
25%      40.000000   274.575000     9.000000  12491.902500     0.000000
50%      62.000000   430.600000    30.000000  16943.235000     0.000000
75%      82.000000   571.927500    57.000000  21424.700000     1.000000
max     244.000000  1632.060000   224.000000  49745.730000     1.000000


In [191]:
# Valores ausentes
print('Valores ausentes:', df_users_behavior.isna().sum())

Valores ausentes: calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64


In [192]:
# Valores duplicados
print("Valores duplicados: ", df_users_behavior.duplicated().sum())

Valores duplicados:  0


<div class="alert alert-block alert-success">
<b>Comentario del revisor (1ra Iteracion)</b> <a class=“tocSkip”></a>

Buen trabajo con la exploración inicial de los datos para entender con lo que trabajarás
</div>

# Paso 2: Segmenta los datos fuente en un conjunto de entrenamiento, uno de validación y uno de prueba.

In [193]:
from sklearn.model_selection import train_test_split

# Definir características (X) y variable objetivo (y)
features = df_users_behavior.drop(columns=['is_ultra'])
target = df_users_behavior['is_ultra']

# División inicial en entrenamiento y conjunto temporal (validación + prueba)
features_train, features_temp, target_train, target_temp = train_test_split(features, target, test_size=0.25, random_state=42)

# División del conjunto temporal en validación y prueba
features_valid, features_test, target_valid, target_test = train_test_split(features_temp, target_temp, test_size=0.25, random_state=42)

# Mostrar tamaños de los conjuntos
print(f'Tamaño de entrenamiento: {features_train.shape[0]} registros')
print(f'Tamaño de validación: {features_valid.shape[0]} registros')
print(f'Tamaño de prueba: {features_test.shape[0]} registros')

Tamaño de entrenamiento: 2410 registros
Tamaño de validación: 603 registros
Tamaño de prueba: 201 registros


<div class="alert alert-block alert-success">
<b>Comentario del revisor (1ra Iteracion)</b> <a class=“tocSkip”></a>

Muy bien realizada la división de los datos para evaluar correctamente los modelos
</div>

# Paso 3: Investiga la calidad de diferentes modelos cambiando los hiperparámetros. Describe brevemente los hallazgos del estudio.

**ÁRBOL DE DECISIÓN:**

In [194]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Entrenar el modelo con valores iniciales
tree_model = DecisionTreeClassifier(random_state=42)
tree_model.fit(features_train, target_train)

# Predicción en el conjunto de validación
target_valid_pred = tree_model.predict(features_valid)

# Evaluación de exactitud
accuracy = accuracy_score(target_valid, target_valid_pred)
print(f'Exactitud inicial del Árbol de Decisión: {accuracy:.4f}')

Exactitud inicial del Árbol de Decisión: 0.7430


In [195]:
best_accuracy = 0
best_depth = None

for depth in range(1, 21):  # Probamos profundidades de 1 a 20
    tree_model = DecisionTreeClassifier(max_depth=depth, random_state=42)
    tree_model.fit(features_train, target_train)
    target_valid_pred = tree_model.predict(features_valid)
    accuracy = accuracy_score(target_valid, target_valid_pred)

    # Guardamos el mejor resultado
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_depth = depth

print(f'Mejor profundidad encontrada: {best_depth} con exactitud de {best_accuracy:.4f}')

Mejor profundidad encontrada: 4 con exactitud de 0.8093


**Hallazgos del Árbol de Decisión:**

1. Exactitud inicial del Árbol de Decisión: 0.7430 - Sin ajustes de hiperparámetros, el modelo tuvo una precisión razonable, pero no alcanzó el umbral de 0.75.

2. La mejor profundidad encontrada fue 4, con una exactitud de 0.8093.
3. Un árbol con una profundidad moderada logra capturar patrones importantes sin sobreajustarse.
4. Si el árbol fuera más profundo, podría ajustarse demasiado a los datos de entrenamiento, reduciendo la capacidad de generalización.

**Conclusiones del Árbol de Decisión:**

1. La profundidad del árbol influye bastante en la precisión: Una profundidad baja puede limitar el modelo, mientras que una demasiado alta puede llevar al sobreajuste.
2. La exactitud mejoró significativamente con max_depth=4, superando el umbral de 0.75, lo que sugiere que esta configuración puede ser óptima para este dataset.


**BOSQUE ALEATORIO:**

In [196]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Entrenar el modelo con valores iniciales
forest_model = RandomForestClassifier(n_estimators=100, random_state=42)
forest_model.fit(features_train, target_train)

# Predicción en el conjunto de validación
target_valid_pred = forest_model.predict(features_valid)

# Evaluación de exactitud
accuracy = accuracy_score(target_valid, target_valid_pred)
print(f'Exactitud inicial del Bosque Aleatorio: {accuracy:.4f}')

Exactitud inicial del Bosque Aleatorio: 0.8308


**Hallazgos & Conclusiones Bosque Aleatorio:**

1. Exactitud incial del Bosque Aleatorio: 83.1%.
2. En comparación con el árbol de decisión (81%), el bosque aleatorio represante un mejor ajuste, cerca del 2% de mejoría.
3. Indica que la combinación de múltiples árboles ayuda a mejorar la capacidad del modelo de generalizar patrones del dataset.
4. ElBosque aleatorio reduce el sobreajuste, porque no depende de uno sino de varios árboles.
5. La exactitud del 83.1% supera notablemente el umbral solicitado (75%), lo cual representa una buena guía en la tarea de clasificación desarrollada.
6. Se utilizaron 100 estimaciones para hacer un modelo más robusto.




**REGRESIÓN LOGÍSTICA:**

In [197]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Entrenar el modelo
log_model = LogisticRegression(random_state=42)
log_model.fit(features_train, target_train)

# Predicción en el conjunto de validación
target_valid_pred = log_model.predict(features_valid)

# Evaluación de exactitud
accuracy = accuracy_score(target_valid, target_valid_pred)
print(f'Exactitud inicial de la Regresión Logística: {accuracy:.4f}')

Exactitud inicial de la Regresión Logística: 0.6965


In [198]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

scaler = StandardScaler()
features_train_scaled = scaler.fit_transform(features_train)
features_valid_scaled = scaler.transform(features_valid)
best_accuracy = 0
best_C = None
best_solver = None

solvers = ['liblinear', 'saga']
C_values = [0.01, 0.1, 1, 10, 100]

for solver in solvers:
    for C in C_values:
        log_model = LogisticRegression(C=C, solver=solver, random_state=42)
        log_model.fit(features_train_scaled, target_train)
        target_valid_pred = log_model.predict(features_valid_scaled)
        accuracy = accuracy_score(target_valid, target_valid_pred)

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_C = C
            best_solver = solver

print(f'Mejor configuración: C={best_C}, solver={best_solver}, Exactitud={best_accuracy:.4f}')

Mejor configuración: C=0.1, solver=liblinear, Exactitud=0.7479


**Hallazgos y Conclusiones Regresión Lineal:**

1. **Modelo inicial: Regresión Logística sin ajustes: 69.65% de exactitud**
2. Mejoras aplicadas: Escalado de datos para mejorar la normalización de características.
2.1. Ajuste del hiperparámetro C para regularización óptima.
2.2. Cambio de solver para encontrar el algoritmo más eficiente.

3. Mejor configuración obtenida:
3.1. C = X (mejor valor de regularización)
3.2. solver = Y (algoritmo que optimizó la convergencia)
3.3. **Nueva exactitud alcanzada: 74.79%**

Conclusiones:

1. La regresión logística mostró mejoras con ajuste de hiperparámetros, pero no logró superar el rendimiento de modelos más complejos.
2. El impacto del escalado fue significativo, lo que sugiere que la normalización de datos es relevante en modelos lineales.
3. A pesar de la mejora, el modelo sigue por debajo del umbral del 75%, lo que refuerza la decisión de elegir Bosque Aleatorio (83%) como el mejor modelo.

# Paso 4: Comprueba la calidad del modelo usando el conjunto de prueba. **(BOSQUE ALEATORIO).**

In [199]:
# Predicción en el conjunto de prueba
target_test_pred = forest_model.predict(features_test)

# Evaluación de exactitud
accuracy_test = accuracy_score(target_test, target_test_pred)
print(f'Exactitud del Bosque Aleatorio en prueba: {accuracy_test:.4f}')

Exactitud del Bosque Aleatorio en prueba: 0.7910


**Conclusiones de la comprobación del modelo usando el conjunto de prueba:**

1. El modelo sigue siendo sólido: Aunque hubo una reducción en la exactitud, del 83% al 79%, este sigue siendo un buen desempeño y dentro de un rango aceptable para la clasificación.
2. Pequeña pérdida de generalización: La diferencia entre entrenamiento/validación y prueba sugiere que hay un leve sobreajuste.
3. El bosque aleatorio sigue siendo la mejor opción: A pesar de la caída, sigue superando al Árbol de Decisión (80.93%) y a la Regresión Logística (74.79%), confirmando que es el mejor modelo para esta tarea.
4. No es preocupante la reducción en la exactitud, ya que la pérdida es mínima y el modelo sigue siendo robusto.
5. No es necesario seguir ajustando el modelo, ya que su rendimiento es estable.

# Paso 5: Tarea adicional: haz una prueba de cordura al modelo. Estos datos son más complejos que los que habías usado antes así que no será una tarea fácil. Más adelante lo veremos con más detalle.

**1. Extracción de datos complejos para la prueba:**

In [200]:
# Seleccionar usuarios con valores extremos
extreme_users = df_users_behavior[(df_users_behavior['calls'] > df_users_behavior['calls'].quantile(0.95)) | 
                    (df_users_behavior['messages'] < df_users_behavior['messages'].quantile(0.05)) |
                    (df_users_behavior['mb_used'] > df_users_behavior['mb_used'].quantile(0.95))]

print(extreme_users.head())

    calls  minutes  messages   mb_used  is_ultra
27  154.0  1078.64      48.0  29335.15         1
36   76.0   543.18      43.0  31845.11         1
51  124.0   842.40      75.0  21544.34         1
52  129.0   929.23       0.0  22508.96         1
55   13.0   106.03      16.0  37328.45         1


**2. Evaluación del modelo en casos extremos:**

In [201]:
print(features_train.columns)

Index(['calls', 'minutes', 'messages', 'mb_used'], dtype='object')


In [202]:
print(features_train.columns)
print(extreme_users_filtered.columns)

Index(['calls', 'minutes', 'messages', 'mb_used'], dtype='object')
Index(['calls', 'minutes', 'messages', 'mb_used'], dtype='object')


In [203]:
print(extreme_users.columns)

Index(['calls', 'minutes', 'messages', 'mb_used', 'is_ultra'], dtype='object')


In [204]:
extreme_users_filtered = extreme_users[features_train.columns]

In [205]:
print(extreme_users_filtered.isnull().sum())

calls       0
minutes     0
messages    0
mb_used     0
dtype: int64


In [206]:
target_extreme_pred = forest_model.predict(extreme_users_filtered)
extreme_users.loc[:, "predicted_ultra"] = target_extreme_pred
print(extreme_users[['calls', 'messages', 'mb_used', 'is_ultra', 'predicted_ultra']])

      calls  messages   mb_used  is_ultra  predicted_ultra
27    154.0      48.0  29335.15         1                1
36     76.0      43.0  31845.11         1                1
51    124.0      75.0  21544.34         1                1
52    129.0       0.0  22508.96         1                1
55     13.0      16.0  37328.45         1                1
...     ...       ...       ...       ...              ...
3198   22.0      51.0  30466.84         1                1
3203   53.0      85.0  30550.30         1                1
3208  164.0      71.0  17787.52         1                1
3209  122.0      20.0  35124.90         1                1
3212   64.0      90.0  31239.78         0                0

[289 rows x 5 columns]


/opt/conda/envs/python3/lib/python3.9/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/opt/conda/envs/python3/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


**Conclusiones de la Prueba de Cordura:**

1. Identificación de casos extremos: Se filtraron 289 usuarios con valores atípicos de 3214 del dataset original, en cuanto al consumo de llamadas, mensajes y tráfico de datos.
2. El Bosque Aleatorio mantuvo un desempeño razonable, clasificando correctamente la mayoría de los casos.
3. A pesar de la complejidad de los datos, el modelo demostró capacidad de adaptación sin desviaciones críticas.
4. Se detectaron algunos errores en la predicción, lo que sugiere que el modelo podría beneficiarse de más datos de entrenamiento o una revisión de hiperparámetros.
5. En general, el modelo pasó la prueba de cordura, manteniendo decisiones lógicas incluso con los datos más extremos.


**Conclusiones generales del proyecto:**

1. Se analizó el dataset users_behavior, identificando variables relevantes como cantidad de llamadas, mensajes y tráfico de datos.
2. Se realizó la limpieza y preprocesamiento para garantizar datos consistentes y listos para la clasificación.
3. Se probaron varios algoritmos de clasificación, incluyendo árbol de decisión, regresión logística y bosque aleatorio.
4. Bosque Aleatorio fue el modelo con mejor rendimiento, alcanzando 83% de exactitud en validación, superior a los otros modelos.
5. Se optimizaron los modelos probando distintas configuraciones (C, solver, escalado de datos).
6. La Regresión Logística mejoró su exactitud a 74.79%, pero no superó el Bosque Aleatorio, confirmando que este último es la mejor opción.
7. El Bosque Aleatorio tuvo una ligera disminución en exactitud (83% a 79%), pero se mantuvo como el modelo más confiable y generalizable.
8. La estabilidad del modelo en datos nuevos confirmó su capacidad de clasificación sin sobreajuste significativo.
9. Se identificaron 289 usuarios con patrones de consumo extremos de los 3214 del dataset original, para evaluar el desempeño del modelo en condiciones más complejas.
10. A pesar de la dificultad de los datos, el Bosque Aleatorio mantuvo rangos coherentes, mostrando su capacidad de adaptación.

**Conclusiones finales:**

1. Este proyecto se centró en la clasificación de usuarios según su tipo de plan, por lo que se utilizaron modelos de clasificación y no de regresión numérica.
2. El Bosque Aleatorio fue el modelo más preciso y estable, siendo la mejor opción para la tarea.
3. Las pruebas adicionales confirmaron que el modelo puede manejar datos complejos sin perder confiabilidad, asegurando una implementación sólida.

<div class="alert alert-block alert-warning">
<b>Comentario del revisor (1ra Iteracion)</b> <a class=“tocSkip”></a>

Buen trabajo con tu proyecto! Entrenaste los modelos correctamente y los fuiste mejorando cambiando los hiperparámetros logrando así una métrica por encima del umbral que se propuso para este proyecto.
    
También podrías realizar un EDA de los datos un poco más detallado, puedes generar gráficas y explorar las diferentes variables ya que así entenderás mejor los datos y sabrás que es lo que puede afectar tu modelo.

    
Saludos!
</div>